In [1]:
import pandas as pd
from dotenv import load_dotenv
import os
from yt_comment_scraper_util import YoutubeCommentScraper, CommentScraperUtil
from datetime import datetime
load_dotenv()
import csv

In [2]:
api_key = os.getenv('YOUTUBE_API_3')  # make sure you have your API key in the .env file
scraper = YoutubeCommentScraper(api_key)

In [ ]:
start_date = '2024-01-01'
channel = 'CNN'
scrape_iteration = 1

script_dir = os.getcwd()

path_to_link_csv = os.path.join(script_dir,f'../data/Raw Data/{channel}_links.csv')
comment_csv_load_path = os.path.join(script_dir,f'../data/Raw Data/{channel}_comments_{str(scrape_iteration)}.csv')
comment_csv_save_path = os.path.join(script_dir,f'../data/Raw Data/{channel}_comments_{str(scrape_iteration+1)}.csv')
path_to_error_logs = os.path.join(script_dir,f'../data/Error Logs/{channel}_{str(datetime.now())}.csv')


most_recent_video_id, previous_comments_df = CommentScraperUtil.get_earliest_video_id(comment_csv_load_path)
video_ids_to_scrape = CommentScraperUtil.load_video_ids(path_to_link_csv, start_date, most_recent_video_id)


In [4]:
all_comments = []
errors = []
for count, video_id in enumerate(video_ids_to_scrape):
    print(f'Scraping video #{count} | Video Id: {video_id}')
    test = scraper.get_video_comments(video_id)
    comments_data, error =test
    if comments_data:
        all_comments.extend(comments_data)
    if error:
        print(f'Error for {video_id}')
        errors.append({'video_id': video_id, "error": error})
        if error ==  "Quota Exceeded":
            break
new_comments = pd.DataFrame(all_comments)
errors_df = pd.DataFrame(errors)

Scraping video #0 | Video Id: 1E6YHpjwvmw
Scraping video #1 | Video Id: xXdZEG2ISj4
Scraping video #2 | Video Id: XzAzHijMRbk
Scraping video #3 | Video Id: XRp6JhtOh8s
Scraping video #4 | Video Id: mumxaYErDAs
Scraping video #5 | Video Id: y-LXLMj8sRc
Scraping video #6 | Video Id: Lke0vZt-Uns
Scraping video #7 | Video Id: jYBmi7l0i2A
Scraping video #8 | Video Id: 5dpqldviiwo
Scraping video #9 | Video Id: 8u0PkwYBRuU
Scraping video #10 | Video Id: UD0Zh-Mh67I
Scraping video #11 | Video Id: _sGHfmCnB5M
Scraping video #12 | Video Id: wbGaR3BmAzo
Scraping video #13 | Video Id: KprBTnLQ7yc
Scraping video #14 | Video Id: gdx2_R7NbpE
Scraping video #15 | Video Id: xD7YcTYai7M
Scraping video #16 | Video Id: jgOTLU4djGg
Scraping video #17 | Video Id: Sz2vmrBogWU
Scraping video #18 | Video Id: UCpBNjM-I9U
Scraping video #19 | Video Id: J9oVfXXJY0s
Scraping video #20 | Video Id: lptYkKQnBW0
Scraping video #21 | Video Id: nTw3Rc73GuQ
Scraping video #22 | Video Id: rLqvlGTHrrk
Scraping video #23 | 

In [ ]:
comments_df = new_comments
comments_df.to_csv(comment_csv_save_path, quoting=csv.QUOTE_NONNUMERIC, escapechar='\\', index=False, encoding='utf-8')
errors_df.to_csv(path_to_error_logs, index=False)
